<a href="https://colab.research.google.com/github/elrrascabuches/Talleres-IA-aplicada-Manuel-Alarcon/blob/main/taller%202%20%3ABi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/LinaMariaCastro/curso-ia-para-economia/blob/main/clases/4_Aprendizaje_no_supervisado/2_Solucion_Taller_Apriori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# **Inteligencia Artificial con Aplicaciones en Economía I**

- 👩‍🏫 **Profesora:** [Lina María Castro](https://www.linkedin.com/in/lina-maria-castro)  
- 📧 **Email:** [lmcastroco@gmail.com](mailto:lmcastroco@gmail.com)  
- 🎓 **Universidad:** Universidad Externado de Colombia - Facultad de Economía

# **Taller: Análisis de Patrones de Consumo Internacional con Apriori**

**IMPORTANTE**: Guarda una copia de este notebook en tu Google Drive o computador.

**Taller en parejas**

**Nombres estudiantes:**

-
-

**Forma de entrega**

Jupyter Notebook publicado en su cuenta de Github con el nombre “Taller_Apriori_apellidos_estudiantes.ipynb”.

**Plazo de entrega**

Hoy, 16 de octubre, máximo a las 11:59 p.m., debes enviar link del notebook al correo lina.castro6@uexternado.edu.co, de lo contrario, no será tenido en cuenta.

**Instrucciones Generales**

Completa el código en las celdas marcadas con `### TU CÓDIGO AQUÍ ###`. Puedes añadir más celdas si lo requieres.

**Caso de Estudio: Consultoría para el PNUD**

**Contexto:** Una firma multinacional de e-commerce, "Global Retail Inc.", te ha contratado como consultor de datos. La empresa opera en múltiples países y ha notado que sus ventas y la efectividad de sus campañas de marketing varían significativamente entre regiones. Su hipótesis es que los patrones de compra y las asociaciones de productos son diferentes en cada mercado.

**Tu Misión:** Analizar el historial de transacciones de la empresa para descubrir y comparar las reglas de asociación de productos para dos de sus mercados más grandes: el Reino Unido y Alemania. Tu objetivo final es entregar recomendaciones de negocio accionables (ej. estrategias de cross-selling, promociones personalizadas) basadas en los patrones de consumo que descubras en cada país.

**Dataset:** Online Retail Data Set del repositorio de UCI. Encuentra mayor información en: https://archive.ics.uci.edu/dataset/352/online+retail

## Ejercicio 1: Configuración Inicial, Carga y Exploración de Datos

1.1 Importa las librerías necesarias

In [ ]:
### TU CÓDIGO AQUÍ ###
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Configuraciones de visualización
pd.options.display.max_columns = None
pd.options.display.float_format = '{:,.2f}'.format

1.2 Carga el dataset "retail_top30_products.csv" que se encuentra en el repositorio del curso, carpeta "datasets".

In [ ]:
datasate=pd.read_csv('/content/retail_top30_products.csv')

In [ ]:
print("datasate")
print(datasate.shape)

datasate
(40530, 8)


In [ ]:
print("\nInformación general del DataFrame:")
datasate.info()


Información general del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40530 entries, 0 to 40529
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   InvoiceNo    40530 non-null  object 
 1   StockCode    40530 non-null  object 
 2   Description  40530 non-null  object 
 3   Quantity     40530 non-null  int64  
 4   InvoiceDate  40530 non-null  object 
 5   UnitPrice    40530 non-null  float64
 6   CustomerID   33004 non-null  float64
 7   Country      40530 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 2.5+ MB


1.3 Revisa si hay valores nulos en alguna columna y cuántos son

In [ ]:
if datasate.isnull().any().any():
    print("Hay valores nulos en el DataFrame.")
else:
    print("No hay valores nulos en el DataFrame.")
print("la cantidad de nulos es:")
datasate.isnull().sum()

Hay valores nulos en el DataFrame.
la cantidad de nulos es:


,0
InvoiceNo,0
StockCode,0
Description,0
Quantity,0
InvoiceDate,0
UnitPrice,0
CustomerID,7526
Country,0


1.4 Genera las estadísticas descriptivas de las variables numéricas

In [ ]:
### TU CÓDIGO AQUÍ ###
datasate.describe()

,Quantity,UnitPrice,CustomerID
count,"40,530.00","40,530.00","33,004.00"
mean,11.38,4.64,"15,225.76"
std,40.69,57.76,"1,729.80"
min,"-1,930.00",0.00,"12,347.00"
25%,2.00,1.65,"13,767.00"
50%,5.00,2.95,"15,080.00"
75%,10.00,4.95,"16,745.00"
max,"2,880.00","8,142.75","18,283.00"


1.5 Observando las salidas del ejercicio anterior, ¿qué problemas potenciales identificas en las columnas Description, CustomerID y Quantity? ¿Ves algo inusual en los precios (UnitPrice)?

# RTA
En cuanto a cutomer ID podemos decir que tenemos problemas en cuanto a los datos porque no están completos al observarse valores nulos, hay 33004 clientes pero el dataset tiene 40530 filas.

Se puede decir que para la variable quantity hay datos atípicos ya que la media y los cuartiles son valores muy pequeños a comparacion del maximo y muy grandes a comparacion del minimo, que ademas es un valor negativo.

En cuanto a UnitPrice resulta extraño tener un precio de cero que ademas es el precio minimo, tambien que exista un precio maximo de mas de ocho mil cuando el resto d eprecios parece ubicarse en rangos pequeños

Podemos observar que en la columna no parece tener problemas potenciales dado que tiene datos completos y no repetidos, para verificarlo notese que:

In [ ]:
print('demos un visstazo a los datos')
datasate.head()

demos un visstazo a los datos


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,"17,850.00",United Kingdom
1,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,"13,047.00",United Kingdom
2,536368,22960,JAM MAKING SET WITH JARS,6,2010-12-01 08:34:00,4.25,"13,047.00",United Kingdom
3,536370,22727,ALARM CLOCK BAKELIKE RED,24,2010-12-01 08:45:00,3.75,"12,583.00",France
4,536370,POST,POSTAGE,3,2010-12-01 08:45:00,18.00,"12,583.00",France


In [ ]:
print('los productos son los siguiente y no están repetidos')
datasate['Description'].unique()

los productos son los siguiente y no están repetidos


array(['WHITE HANGING HEART T-LIGHT HOLDER',
       'ASSORTED COLOUR BIRD ORNAMENT', 'JAM MAKING SET WITH JARS',
       'ALARM CLOCK BAKELIKE RED ', 'POSTAGE',
       "PAPER CHAIN KIT 50'S CHRISTMAS ",
       'WOODEN PICTURE FRAME WHITE FINISH', 'JUMBO BAG PINK POLKADOT',
       'LUNCH BAG RED RETROSPOT', 'PACK OF 72 RETROSPOT CAKE CASES',
       'JUMBO STORAGE BAG SUKI', 'JAM MAKING SET PRINTED',
       'JUMBO SHOPPER VINTAGE RED PAISLEY',
       'NATURAL SLATE HEART CHALKBOARD ', 'HEART OF WICKER SMALL',
       'JUMBO BAG RED RETROSPOT', 'LUNCH BAG SPACEBOY DESIGN ',
       'LUNCH BAG PINK POLKADOT', 'LUNCH BAG CARS BLUE',
       'LUNCH BAG  BLACK SKULL.', 'REGENCY CAKESTAND 3 TIER',
       'ROSES REGENCY TEACUP AND SAUCER ',
       'GREEN REGENCY TEACUP AND SAUCER',
       'RECIPE BOX PANTRY YELLOW DESIGN', 'PARTY BUNTING',
       'SET OF 3 CAKE TINS PANTRY DESIGN ',
       'SET OF 4 PANTRY JELLY MOULDS', 'LUNCH BAG SUKI DESIGN ',
       'LUNCH BAG APPLE DESIGN', 'SPOTTY BUNTING'], 

## Ejercicio 2: Limpieza y Preprocesamiento de Datos

Los datos del mundo real rara vez son perfectos. Antes de cualquier análisis, debemos "sanear" nuestro dataset. Completa el código en cada paso según las instrucciones.

2.1 **Manejo de Valores Nulos**: Las transacciones sin un CustomerID no son útiles para nosotros, ya que no podemos agrupar las compras de un cliente específico.

In [ ]:
# TAREA: Elimina todas las filas donde 'CustomerID' es nulo.
### TU CÓDIGO AQUÍ ###
datasate.dropna(subset=['CustomerID'], inplace=True)

In [ ]:
# El tipo de dato de CustomerID debe ser entero
### TU CÓDIGO AQUÍ ###
datasate['CustomerID'].astype(int)

,CustomerID
0,17850
1,13047
2,13047
3,12583
4,12583
...,...
40525,17581
40526,13777
40527,15804
40528,15804


2.2 **Limpieza de Descripciones de Productos** Las descripciones pueden tener espacios en blanco al inicio o al final que podrían hacer que un mismo producto se cuente como dos diferentes.

In [ ]:
# TAREA: Limpia la columna 'Description' eliminando espacios extra al inicio y al final.
### TU CÓDIGO AQUÍ ###
df_limpio['Description'] = df_limpio['Description'].str.strip()

2.3 **Filtrado de Transacciones Anómalas**: Las facturas (InvoiceNo) que empiezan con 'C' indican una cancelación. Estas no son compras reales y deben ser eliminadas. Del mismo modo, las cantidades (Quantity) negativas representan devoluciones.

In [ ]:
# TAREA: Elimina las filas que correspondan a cancelaciones.
### TU CÓDIGO AQUÍ ###


In [ ]:
df_limpio = datasate.copy()

In [ ]:
df_limpio = df_limpio[~df_limpio['InvoiceNo'].str.startswith('C')]

In [ ]:
# TAREA: Elimina las filas con cantidades negativas.
### TU CÓDIGO AQUÍ ###
df_limpio = df_limpio[df_limpio['Quantity'] > 0]

In [ ]:
# NO MODIFICAR ESTA CELDA
assert df_limpio['Quantity'].min() > 0, "Error: Todavía hay cantidades negativas."
assert df_limpio['CustomerID'].isnull().sum() == 0, "Error: Aún hay valores nulos en CustomerID."
assert df_limpio[df_limpio['InvoiceNo'].str.contains('C')].empty, "Error: Todavía hay facturas de cancelación."
print("¡La limpieza básica ha sido exitosa!")

¡La limpieza básica ha sido exitosa!


## Ejercicio 3: Análisis Comparativo por País

Ahora que los datos están limpios, vamos a segmentarlos y a aplicar el algoritmo Apriori para encontrar los patrones de compra en el Reino Unido y Alemania.

**Preparación de la Cesta de Mercado (Función)**

La siguiente función toma un dataframe, lo agrupa por factura y descripción, y lo transforma en el formato de matriz binaria que necesita el algoritmo Apriori. Estudia esta función, no necesitas modificarla.

In [ ]:
# NO MODIFICAR ESTA CELDA
def preparar_cesta(dataframe, pais):
    """Filtra por país y prepara la matriz de transacciones."""

    # Filtrar por el país de interés
    df_pais = dataframe[dataframe['Country'] == pais]

    # Crear la cesta: agrupar productos por factura
    cesta = (df_pais.groupby(['InvoiceNo', 'Description'])['Quantity']
             .sum().unstack().reset_index().fillna(0)
             .set_index('InvoiceNo'))

    # Convertir todas las cantidades positivas a 1 y todo lo demás a 0
    def encode_units(x):
        if x <= 0:
            return 0
        if x >= 1:
            return 1

    cesta_encoded = cesta.applymap(encode_units)
    return cesta_encoded

3.1 Análisis para el Reino Unido (United Kingdom)

In [ ]:
df_transacciones = preparar_cesta(df_limpio, 'United Kingdom')

In [ ]:
# TAREA: Usa la función preparar_cesta para obtener la matriz de transacciones del Reino Unido.
### TU CÓDIGO AQUÍ ###


In [ ]:
# TAREA: Aplica el algoritmo apriori para encontrar itemsets con un soporte mínimo de 2%.
### TU CÓDIGO AQUÍ ###

itemsets_frecuentes = apriori(
    df_transacciones,
    min_support=0.02,
    use_colnames=True
)

print("Itemsets Frecuentes (Soporte >= 2%):")
print(itemsets_frecuentes.sort_values(by='support', ascending=False))

Itemsets Frecuentes (Soporte >= 2%):
    support                                           itemsets
27     0.19               (WHITE HANGING HEART T-LIGHT HOLDER)
7      0.14                          (JUMBO BAG RED RETROSPOT)
22     0.14                         (REGENCY CAKESTAND 3 TIER)
1      0.13                    (ASSORTED COLOUR BIRD ORNAMENT)
20     0.13                                    (PARTY BUNTING)
..      ...                                                ...
85     0.02  (LUNCH BAG SPACEBOY DESIGN , LUNCH BAG CARS BL...
94     0.02  (LUNCH BAG SPACEBOY DESIGN , LUNCH BAG CARS BL...
92     0.02  (LUNCH BAG SUKI DESIGN , LUNCH BAG SPACEBOY DE...
35     0.02  (SET OF 3 CAKE TINS PANTRY DESIGN , JAM MAKING...
29     0.02  (REGENCY CAKESTAND 3 TIER, ASSORTED COLOUR BIR...

[100 rows x 2 columns]


In [ ]:
# TAREA: Genera las reglas de asociación. Queremos reglas con un Lift mayor a 3.
### TU CÓDIGO AQUÍ ###
rules_uk = association_rules(itemsets_frecuentes, metric="lift", min_threshold=3)

In [ ]:
# Ordena las reglas por Lift y Confianza de mayor a menor
### TU CÓDIGO AQUÍ ###
rules_uk = rules_uk.sort_values(['lift', 'confidence'], ascending=[False, False])
print("Top 10 Association Rules for United Kingdom (Lift > 3):")
display(rules_uk.head(10))

Top 10 Association Rules for United Kingdom (Lift > 3):


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
71,"(ROSES REGENCY TEACUP AND SAUCER , REGENCY CAK...",(GREEN REGENCY TEACUP AND SAUCER),0.04,0.06,0.03,0.75,12.37,1.00,0.02,3.80,0.95,0.38,0.74,0.59
74,(GREEN REGENCY TEACUP AND SAUCER),"(ROSES REGENCY TEACUP AND SAUCER , REGENCY CAK...",0.06,0.04,0.03,0.43,12.37,1.00,0.02,1.70,0.98,0.38,0.41,0.59
72,"(REGENCY CAKESTAND 3 TIER, GREEN REGENCY TEACU...",(ROSES REGENCY TEACUP AND SAUCER ),0.03,0.07,0.03,0.83,12.24,1.00,0.02,5.35,0.95,0.36,0.81,0.61
73,(ROSES REGENCY TEACUP AND SAUCER ),"(REGENCY CAKESTAND 3 TIER, GREEN REGENCY TEACU...",0.07,0.03,0.03,0.39,12.24,1.00,0.02,1.59,0.98,0.36,0.37,0.61
2,(ROSES REGENCY TEACUP AND SAUCER ),(GREEN REGENCY TEACUP AND SAUCER),0.07,0.06,0.05,0.70,11.54,1.00,0.04,3.15,0.98,0.58,0.68,0.74
3,(GREEN REGENCY TEACUP AND SAUCER),(ROSES REGENCY TEACUP AND SAUCER ),0.06,0.07,0.05,0.78,11.54,1.00,0.04,4.20,0.97,0.58,0.76,0.74
83,"(LUNCH BAG CARS BLUE, LUNCH BAG BLACK SKULL.)",(LUNCH BAG PINK POLKADOT),0.04,0.08,0.02,0.62,7.37,1.00,0.02,2.42,0.90,0.25,0.59,0.46
86,(LUNCH BAG PINK POLKADOT),"(LUNCH BAG CARS BLUE, LUNCH BAG BLACK SKULL.)",0.08,0.04,0.02,0.29,7.37,1.00,0.02,1.36,0.94,0.25,0.27,0.46
146,(LUNCH BAG PINK POLKADOT),"(LUNCH BAG CARS BLUE, LUNCH BAG RED RETROSPOT)",0.08,0.04,0.03,0.30,7.29,1.00,0.02,1.37,0.94,0.25,0.27,0.46
143,"(LUNCH BAG CARS BLUE, LUNCH BAG RED RETROSPOT)",(LUNCH BAG PINK POLKADOT),0.04,0.08,0.03,0.62,7.29,1.00,0.02,2.38,0.90,0.25,0.58,0.46


3.3 Observa las 3 reglas con el Lift más alto para el Reino Unido (1, 3 y 5). **Interprétalas:** ¿Qué te dicen estas asociaciones? ¿Qué tipo de productos son?

3.4 Interpreta el Soporte para el antecedente y el consecuente, la Confianza y el Lift

3.5 **Recomendación de Negocio:** Basado en estas reglas, ¿qué promoción o estrategia de venta específica podrías sugerir para el mercado del Reino Unido?

3.6 Análisis para Alemania (Germany)

In [ ]:
# TAREA: Usa la función preparar_cesta para obtener la matriz de transacciones de Alemania.
### TU CÓDIGO AQUÍ ###
df_transacciones = preparar_cesta(df_limpio, 'Germany')

In [ ]:
# TAREA: Aplica el algoritmo apriori con un soporte mínimo del 2%.
### TU CÓDIGO AQUÍ ###

itemsets_Alemania = apriori(
    df_transacciones,
    min_support=0.02,
    use_colnames=True
)

print("Itemsets Alemania (Soporte >= 2%):")
print(itemsets_Alemania.sort_values(by='support', ascending=False))

Itemsets Alemania (Soporte >= 2%):
    support                                           itemsets
27     0.19               (WHITE HANGING HEART T-LIGHT HOLDER)
7      0.14                          (JUMBO BAG RED RETROSPOT)
22     0.14                         (REGENCY CAKESTAND 3 TIER)
1      0.13                    (ASSORTED COLOUR BIRD ORNAMENT)
20     0.13                                    (PARTY BUNTING)
..      ...                                                ...
85     0.02  (LUNCH BAG SPACEBOY DESIGN , LUNCH BAG CARS BL...
94     0.02  (LUNCH BAG SPACEBOY DESIGN , LUNCH BAG CARS BL...
92     0.02  (LUNCH BAG SUKI DESIGN , LUNCH BAG SPACEBOY DE...
35     0.02  (SET OF 3 CAKE TINS PANTRY DESIGN , JAM MAKING...
29     0.02  (REGENCY CAKESTAND 3 TIER, ASSORTED COLOUR BIR...

[100 rows x 2 columns]


In [ ]:
# TAREA: Genera las reglas de asociación con un Lift mayor a 3.
### TU CÓDIGO AQUÍ ###
rules_DU = association_rules(itemsets_Alemania, metric="lift", min_threshold=3)

In [ ]:
# Ordena las reglas por Lift y Confianza de mayor a menor
### TU CÓDIGO AQUÍ ###
rules_DU = rules_DU.sort_values(['lift', 'confidence'], ascending=[False, False])
print("Top 10 Association Rules for Germany (Lift > 3):")
display(rules_DU.head(10))

Top 10 Association Rules for Germany (Lift > 3):


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
71,"(ROSES REGENCY TEACUP AND SAUCER , REGENCY CAK...",(GREEN REGENCY TEACUP AND SAUCER),0.04,0.06,0.03,0.75,12.37,1.00,0.02,3.80,0.95,0.38,0.74,0.59
74,(GREEN REGENCY TEACUP AND SAUCER),"(ROSES REGENCY TEACUP AND SAUCER , REGENCY CAK...",0.06,0.04,0.03,0.43,12.37,1.00,0.02,1.70,0.98,0.38,0.41,0.59
72,"(REGENCY CAKESTAND 3 TIER, GREEN REGENCY TEACU...",(ROSES REGENCY TEACUP AND SAUCER ),0.03,0.07,0.03,0.83,12.24,1.00,0.02,5.35,0.95,0.36,0.81,0.61
73,(ROSES REGENCY TEACUP AND SAUCER ),"(REGENCY CAKESTAND 3 TIER, GREEN REGENCY TEACU...",0.07,0.03,0.03,0.39,12.24,1.00,0.02,1.59,0.98,0.36,0.37,0.61
2,(ROSES REGENCY TEACUP AND SAUCER ),(GREEN REGENCY TEACUP AND SAUCER),0.07,0.06,0.05,0.70,11.54,1.00,0.04,3.15,0.98,0.58,0.68,0.74
3,(GREEN REGENCY TEACUP AND SAUCER),(ROSES REGENCY TEACUP AND SAUCER ),0.06,0.07,0.05,0.78,11.54,1.00,0.04,4.20,0.97,0.58,0.76,0.74
83,"(LUNCH BAG CARS BLUE, LUNCH BAG BLACK SKULL.)",(LUNCH BAG PINK POLKADOT),0.04,0.08,0.02,0.62,7.37,1.00,0.02,2.42,0.90,0.25,0.59,0.46
86,(LUNCH BAG PINK POLKADOT),"(LUNCH BAG CARS BLUE, LUNCH BAG BLACK SKULL.)",0.08,0.04,0.02,0.29,7.37,1.00,0.02,1.36,0.94,0.25,0.27,0.46
146,(LUNCH BAG PINK POLKADOT),"(LUNCH BAG CARS BLUE, LUNCH BAG RED RETROSPOT)",0.08,0.04,0.03,0.30,7.29,1.00,0.02,1.37,0.94,0.25,0.27,0.46
143,"(LUNCH BAG CARS BLUE, LUNCH BAG RED RETROSPOT)",(LUNCH BAG PINK POLKADOT),0.04,0.08,0.03,0.62,7.29,1.00,0.02,2.38,0.90,0.25,0.58,0.46


3.7 Observa las 3 reglas con el Lift más alto para Alemania (1, 3 y 5). **Interprétalas:** ¿Qué patrones de consumo específicos del mercado alemán revelan estas reglas? ¿Son diferentes a los del Reino Unido?



3.8 Interpreta el Soporte para el antecedente y el consecuente, la Confianza y el Lift

3.9 **Recomendación de Negocio:** ¿Qué campaña de marketing (diferente a la del Reino Unido) podrías diseñar para los clientes alemanes?